In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.datasets import mnist
import wandb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [4]:
!pip install wandb

In [3]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: viinod9 (viinod9-iitm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
import wandb
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

# Initialize WandB
wandb.init(project="Vinod_FashionMNIST", name="One_Image_Per_Class", reinit=True)

# Load Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Define class labels
fmnist_labels = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'
}

# Select **one unique image per class**
samples = []
for class_idx in range(10):
    index = np.where(y_train == class_idx)[0][0]  # Get first occurrence of class
    image = wandb.Image(x_train[index], caption=fmnist_labels[class_idx])
    samples.append(image)

# Log **all 10 images in one step**
wandb.log({"Fashion MNIST Classes": samples})

# Finish WandB run
wandb.finish()


In [6]:
# Load and preprocess dataset
def Data_Preprocess():
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

    # Split into train and validation
    val_size = 5000
    x_val, y_val = x_train[:val_size], y_train[:val_size]
    x_train, y_train = x_train[val_size:], y_train[val_size:]

    # Normalize dataset
    x_train, x_val, x_test = x_train / 255.0, x_val / 255.0, x_test / 255.0

    # One-hot encoding
    y_train = to_categorical(y_train, 10)
    y_val = to_categorical(y_val, 10)
    y_test = to_categorical(y_test, 10)

    return x_train, y_train, x_val, y_val, x_test, y_test




# Load data
x_train, y_train, x_val, y_val, x_test, y_test = Data_Preprocess()

# Define necessary functions
def Weights_Initialization(num_hidden_layer, num_nodes_hidden_layers, weight, input_size, output_size):
    weights = {}
    prev_size = input_size
    hidden_size = num_nodes_hidden_layers[0] if isinstance(num_nodes_hidden_layers, list) else num_nodes_hidden_layers

    for i in range(num_hidden_layer):
        if weight == 'random':
            weights[f'W{i+1}'] = np.random.randn(prev_size, hidden_size) * 0.01
        elif weight == 'xavier':
            weights[f'W{i+1}'] = np.random.randn(prev_size, hidden_size) * np.sqrt(1 / prev_size)
        weights[f'b{i+1}'] = np.zeros((1, hidden_size))
        prev_size = hidden_size

    if weight == 'random':
        weights['W_out'] = np.random.randn(prev_size, output_size) * 0.01
    elif weight == 'xavier':
        weights['W_out'] = np.random.randn(prev_size, output_size) * np.sqrt(1 / prev_size)
    weights['b_out'] = np.zeros((1, output_size))

    return weights

def Activation_Function(Z, activation):
    if activation == 'sigmoid':
        return 1 / (1 + np.exp(-Z))
    elif activation == 'tanh':
        return np.tanh(Z)
    elif activation == 'relu':
        return np.maximum(0, Z)
    elif activation == 'softmax':
        expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return expZ / np.sum(expZ, axis=1, keepdims=True)

def Derrivative_Activation(Z, activation):
    if activation == 'sigmoid':
        sig = Activation_Function(Z, 'sigmoid')
        return sig * (1 - sig)
    elif activation == 'tanh':
        return 1 - np.tanh(Z)**2
    elif activation == 'relu':
        return (Z > 0).astype(float)

def Cross_Entropy_Loss(y_actual, y_pred):
    return -np.mean(y_actual * np.log(y_pred + 1e-9))


def MSE_Loss(y_actual, y_pred):
    return np.mean((y_actual - y_pred) ** 2)


def Forward_Propogation(X, weights, num_hidden_layer, activation):
    A = X.reshape(X.shape[0], -1)
    cache = {'A0': A}

    for i in range(num_hidden_layer):
        Z = np.dot(A, weights[f'W{i+1}']) + weights[f'b{i+1}']
        A = Activation_Function(Z, activation)
        cache[f'Z{i+1}'] = Z
        cache[f'A{i+1}'] = A

    Z_out = np.dot(A, weights['W_out']) + weights['b_out']
    A_out = Activation_Function(Z_out, 'softmax')

    cache['Z_out'] = Z_out
    cache['A_out'] = A_out

    return A_out, cache

def Back_Propogation(X, y_actual, weights, cache, num_hidden_layer, activation):
    gradients = {}
    m = X.shape[0]

    dZ_out = cache['A_out'] - y_actual
    gradients['dW_out'] = np.dot(cache[f'A{num_hidden_layer}'].T, dZ_out) / m
    gradients['db_out'] = np.sum(dZ_out, axis=0, keepdims=True) / m

    dA = np.dot(dZ_out, weights['W_out'].T)

    for i in range(num_hidden_layer, 0, -1):
        dZ = dA * Derrivative_Activation(cache[f'Z{i}'], activation)
        gradients[f'dW{i}'] = np.dot(cache[f'A{i-1}'].T, dZ) / m
        gradients[f'db{i}'] = np.sum(dZ, axis=0, keepdims=True) / m
        dA = np.dot(dZ, weights[f'W{i}'].T)

    return gradients

def Calculate_Accuracy(X, y_actual, weights, num_hidden_layer, activation):
    y_pred, _ = Forward_Propogation(X, weights, num_hidden_layer, activation)
    return np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_actual, axis=1))

def Stochastic_GD(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, loss_function='cross_entropy'):
    wandb.init(project="stochastic")
    weights = Weights_Initialization(num_hidden_layer, num_nodes_hidden_layers, weight, input_size, output_size)

    for epoch in range(epochs):
        for i in range(0, x_train.shape[0], batch_size):
            X_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]

            y_pred, cache = Forward_Propogation(X_batch, weights, num_hidden_layer, activation)
            gradients = Back_Propogation(X_batch, y_batch, weights, cache, num_hidden_layer, activation)

            for key in weights:
                weights[key] -= lr * gradients[f'd{key}']

        train_acc = Calculate_Accuracy(x_train, y_train, weights, num_hidden_layer, activation)
        val_acc = Calculate_Accuracy(x_val, y_val, weights, num_hidden_layer, activation)

        # Select loss function dynamically
        if loss_function == 'cross_entropy':
            train_loss = Cross_Entropy_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = Cross_Entropy_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])
        elif loss_function == 'mse':
            train_loss = MSE_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = MSE_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])

        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val Loss={val_loss:.4f}")
        wandb.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, "epoch": epoch + 1})

    return weights




# # Set parameters and train
# num_hidden_layer = 4
# num_nodes_hidden_layers = [128]
# weight = 'random'
# input_size = 28 * 28  # Flattened image size
# output_size = 10  # Number of classes
# lr = 0.01
# batch_size = 64
# epochs = 5
# activation = 'sigmoid'

# trained_weights = Stochastic_GD(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, loss_function='cross_entropy')
# wandb.finish()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
# Modify Momentum_GD to use either cross-entropy or MSE loss

def Momentum_GD(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, momentum=0.9, loss_function='cross_entropy'):
    wandb.init(project="momentum")
    weights = Weights_Initialization(num_hidden_layer, num_nodes_hidden_layers, weight, input_size, output_size)
    velocity = {key: np.zeros_like(value) for key, value in weights.items()}

    for epoch in range(epochs):
        for i in range(0, x_train.shape[0], batch_size):
            X_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            y_pred, cache = Forward_Propogation(X_batch, weights, num_hidden_layer, activation)
            gradients = Back_Propogation(X_batch, y_batch, weights, cache, num_hidden_layer, activation)

            for key in weights:
                velocity[key] = momentum * velocity[key] - lr * gradients[f'd{key}']
                weights[key] += velocity[key]

        train_acc = Calculate_Accuracy(x_train, y_train, weights, num_hidden_layer, activation)
        val_acc = Calculate_Accuracy(x_val, y_val, weights, num_hidden_layer, activation)

        # Select loss function dynamically
        if loss_function == 'cross_entropy':
            train_loss = Cross_Entropy_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = Cross_Entropy_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])
        elif loss_function == 'mse':
            train_loss = MSE_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = MSE_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])

        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val Loss={val_loss:.4f}")
        wandb.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, "epoch": epoch + 1})

    return weights

# Example usage


# Set parameters and train
# num_hidden_layer = 3
# num_nodes_hidden_layers = [128]
# weight = 'random'
# input_size = 28 * 28  # Flattened image size
# output_size = 10  # Number of classes
# lr = 0.01
# batch_size = 64
# epochs = 1
# activation = 'sigmoid'
# trained_weights1 = Momentum_GD(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, momentum=0.9, loss_function='cross_entropy')
# wandb.finish()


In [8]:
def Nesterov_GD(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, momentum=0.9, loss_function='cross_entropy'):
    wandb.init(project="nesterov")
    weights = Weights_Initialization(num_hidden_layer, num_nodes_hidden_layers, weight, input_size, output_size)
    velocity = {key: np.zeros_like(value) for key, value in weights.items()}

    for epoch in range(epochs):
        for i in range(0, x_train.shape[0], batch_size):
            temp_weights = {key: weights[key] + momentum * velocity[key] for key in weights}
            X_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            y_pred, cache = Forward_Propogation(X_batch, temp_weights, num_hidden_layer, activation)
            gradients = Back_Propogation(X_batch, y_batch, temp_weights, cache, num_hidden_layer, activation)

            for key in weights:
                velocity[key] = momentum * velocity[key] - lr * gradients[f'd{key}']
                weights[key] += velocity[key]

        train_acc = Calculate_Accuracy(x_train, y_train, weights, num_hidden_layer, activation)
        val_acc = Calculate_Accuracy(x_val, y_val, weights, num_hidden_layer, activation)

        # Select loss function dynamically
        if loss_function == 'cross_entropy':
            train_loss = Cross_Entropy_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = Cross_Entropy_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])
        elif loss_function == 'mse':
            train_loss = MSE_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = MSE_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])

        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val Loss={val_loss:.4f}")
        wandb.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, "epoch": epoch + 1})

    return weights

# Example usage


# # Set parameters and train
# num_hidden_layer = 5
# num_nodes_hidden_layers = [128]
# weight = 'random'
# input_size = 28 * 28  # Flattened image size
# output_size = 10  # Number of classes
# lr = 0.01
# batch_size = 64
# epochs = 1
# activation = 'sigmoid'

# trained_weights2 = Nesterov_GD(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, momentum=0.9, loss_function='cross_entropy')
# wandb.finish()


In [9]:
def RMS_Opt(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, beta=0.9, epsilon=1e-8, loss_function='cross_entropy'):
    wandb.init(project="rmsprop")
    weights = Weights_Initialization(num_hidden_layer, num_nodes_hidden_layers, weight, input_size, output_size)
    cache = {key: np.zeros_like(value) for key, value in weights.items()}

    for epoch in range(epochs):
        for i in range(0, x_train.shape[0], batch_size):
            X_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            y_pred, cache_forward = Forward_Propogation(X_batch, weights, num_hidden_layer, activation)
            gradients = Back_Propogation(X_batch, y_batch, weights, cache_forward, num_hidden_layer, activation)

            for key in weights:
                cache[key] = beta * cache[key] + (1 - beta) * gradients[f'd{key}']**2
                weights[key] -= lr * gradients[f'd{key}'] / (np.sqrt(cache[key]) + epsilon)

        train_acc = Calculate_Accuracy(x_train, y_train, weights, num_hidden_layer, activation)
        val_acc = Calculate_Accuracy(x_val, y_val, weights, num_hidden_layer, activation)

        # Select loss function dynamically
        if loss_function == 'cross_entropy':
            train_loss = Cross_Entropy_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = Cross_Entropy_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])
        elif loss_function == 'mse':
            train_loss = MSE_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = MSE_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])

        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val Loss={val_loss:.4f}")
        wandb.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, "epoch": epoch + 1})

    return weights

# Example usage

# # Set parameters and train
# num_hidden_layer = 4
# num_nodes_hidden_layers = [128]
# weight = 'random'
# input_size = 28 * 28  # Flattened image size
# output_size = 10  # Number of classes
# lr = 0.01
# batch_size = 64
# epochs = 1
# activation = 'sigmoid'


# trained_weights3 = RMS_Opt(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, beta=0.9, epsilon=1e-8, loss_function='cross_entropy')
# wandb.finish()

In [10]:
def Adam_Opt(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, beta1=0.9, beta2=0.999, epsilon=1e-8, loss_function='cross_entropy'):
    wandb.init(project="adam")
    weights = Weights_Initialization(num_hidden_layer, num_nodes_hidden_layers, weight, input_size, output_size)
    m = {key: np.zeros_like(value) for key, value in weights.items()}
    v = {key: np.zeros_like(value) for key, value in weights.items()}

    for epoch in range(epochs):
        for i in range(0, x_train.shape[0], batch_size):
            X_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            y_pred, cache_forward = Forward_Propogation(X_batch, weights, num_hidden_layer, activation)
            gradients = Back_Propogation(X_batch, y_batch, weights, cache_forward, num_hidden_layer, activation)

            for key in weights:
                m[key] = beta1 * m[key] + (1 - beta1) * gradients[f'd{key}']
                v[key] = beta2 * v[key] + (1 - beta2) * (gradients[f'd{key}'] ** 2)
                m_hat = m[key] / (1 - beta1 ** (epoch + 1))
                v_hat = v[key] / (1 - beta2 ** (epoch + 1))
                weights[key] -= lr * m_hat / (np.sqrt(v_hat) + epsilon)

        train_acc = Calculate_Accuracy(x_train, y_train, weights, num_hidden_layer, activation)
        val_acc = Calculate_Accuracy(x_val, y_val, weights, num_hidden_layer, activation)

        # Select loss function dynamically
        if loss_function == 'cross_entropy':
            train_loss = Cross_Entropy_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = Cross_Entropy_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])
        elif loss_function == 'mse':
            train_loss = MSE_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = MSE_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])

        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val Loss={val_loss:.4f}")
        wandb.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, "epoch": epoch + 1})

    return weights

# Example usage

# Set parameters and train
# num_hidden_layer = 4
# num_nodes_hidden_layers = [128]
# weight = 'random'
# input_size = 28 * 28  # Flattened image size
# output_size = 10  # Number of classes
# lr = 0.01
# batch_size = 64
# epochs = 1
# activation = 'sigmoid'


# trained_weights4 = Adam_Opt(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, beta1=0.9, beta2=0.999, epsilon=1e-8, loss_function='cross_entropy')
# wandb.finish()


In [11]:
def NAdam_Opt(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, beta1=0.9, beta2=0.999, epsilon=1e-8, loss_function='cross_entropy'):
    wandb.init(project="nadam")
    weights = Weights_Initialization(num_hidden_layer, num_nodes_hidden_layers, weight, input_size, output_size)
    m = {key: np.zeros_like(value) for key, value in weights.items()}
    v = {key: np.zeros_like(value) for key, value in weights.items()}

    for epoch in range(epochs):
        for i in range(0, x_train.shape[0], batch_size):
            X_batch = x_train[i:i+batch_size]
            y_batch = y_train[i:i+batch_size]
            y_pred, cache_forward = Forward_Propogation(X_batch, weights, num_hidden_layer, activation)
            gradients = Back_Propogation(X_batch, y_batch, weights, cache_forward, num_hidden_layer, activation)

            for key in weights:
                m[key] = beta1 * m[key] + (1 - beta1) * gradients[f'd{key}']
                v[key] = beta2 * v[key] + (1 - beta2) * (gradients[f'd{key}'] ** 2)
                m_hat = m[key] / (1 - beta1 ** (epoch + 1))
                v_hat = v[key] / (1 - beta2 ** (epoch + 1))
                nadam_update = beta1 * m_hat + (1 - beta1) * gradients[f'd{key}']
                weights[key] -= lr * nadam_update / (np.sqrt(v_hat) + epsilon)

        train_acc = Calculate_Accuracy(x_train, y_train, weights, num_hidden_layer, activation)
        val_acc = Calculate_Accuracy(x_val, y_val, weights, num_hidden_layer, activation)

        # Select loss function dynamically
        if loss_function == 'cross_entropy':
            train_loss = Cross_Entropy_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = Cross_Entropy_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])
        elif loss_function == 'mse':
            train_loss = MSE_Loss(y_train, Forward_Propogation(x_train, weights, num_hidden_layer, activation)[0])
            val_loss = MSE_Loss(y_val, Forward_Propogation(x_val, weights, num_hidden_layer, activation)[0])

        print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Train Loss={train_loss:.4f}, Val Acc={val_acc:.4f}, Val Loss={val_loss:.4f}")
        wandb.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss, "epoch": epoch + 1})

    return weights

# Example usage

# Set parameters and train
# num_hidden_layer = 4
# num_nodes_hidden_layers = [128]
# weight = 'random'
# input_size = 28 * 28  # Flattened image size
# output_size = 10  # Number of classes
# lr = 0.01
# batch_size = 64
# epochs = 4
# activation = 'sigmoid'

# trained_weights5 = NAdam_Opt(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, beta1=0.9, beta2=0.999, epsilon=1e-8, loss_function='cross_entropy')
# wandb.finish()


#Question - 4 , 5 , 6

In [26]:

def main():
    sweep_config = {
        'method': 'bayes',
        'metric': {'name': 'accuracy', 'goal': 'maximize'},
        'parameters': {
            'epochs': {'values': [5, 10]},
            'num_layers': {'values': [3, 4, 5]},
            'hidden_size': {'values': [32, 64, 128]},
            'weight_decay': {'values': [0, 0.0005, 0.5]},
            'learning_rate': {'values': [1e-3, 1e-4]},
            'optimizer': {'values': ['stochastic', 'momentum', 'nag', 'rmsprop', 'adam', 'nadam']},
            'batch_size': {'values': [16, 32, 64]},
            'weight_init': {'values': ['random', 'xavier']},
            'activation': {'values': ['sigmoid', 'tanh', 'relu']},
        }
    }
    sweep_id = wandb.sweep(sweep_config, project="Vinod_Assgnment1_Question456")
    wandb.agent(sweep_id, function=train, count=350)

def train():
    wandb.init(project="Vinod_Assignment1_Question456")
    # wandb.init()
    config = wandb.config
    run_name = f"Opt-{config.optimizer}_Layers-{config.num_layers}_HS-{config.hidden_size}_LR-{config.learning_rate}_Batch-{config.batch_size}_Act-{config.activation}"
    wandb.run.name = run_name

    # x_train, y_train, x_val, y_val, _, _ = Data_Preprocess()

    optimizer = config.optimizer

    if optimizer == 'stochastic':
        trained_weights = Stochastic_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')
        # trained_weights = Stochastic_GD(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size)
    elif optimizer == 'momentum':
        trained_weights = Momentum_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')
    elif optimizer == 'nag':
        trained_weights = Nesterov_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')
    elif optimizer == 'rmsprop':
        trained_weights = RMS_Opt(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')
    elif optimizer == 'adam':
        trained_weights = Adam_Opt(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')
    elif optimizer == 'nadam':
        trained_weights = NAdam_Opt(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')

    #wandb.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss})

    wandb.finish()

if __name__ == "__main__":
    main()


Create sweep with ID: bfl29i0j
Sweep URL: https://wandb.ai/viinod9-iitm/Vinod_Assgnment1_Question456/sweeps/bfl29i0j


wandb: Agent Starting Run: j9osbt9i with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 5
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: xavier


Epoch 1: Train Acc=0.8521, Train Loss=0.0421, Val Acc=0.8534, Val Loss=0.0427
Epoch 2: Train Acc=0.8649, Train Loss=0.0378, Val Acc=0.8624, Val Loss=0.0395
Epoch 3: Train Acc=0.8744, Train Loss=0.0350, Val Acc=0.8658, Val Loss=0.0371
Epoch 4: Train Acc=0.8797, Train Loss=0.0336, Val Acc=0.8714, Val Loss=0.0363
Epoch 5: Train Acc=0.8827, Train Loss=0.0329, Val Acc=0.8750, Val Loss=0.0359


epoch,▁▃▅▆█
train_acc,▁▄▆▇█
train_loss,█▅▃▂▁
val_acc,▁▄▅▇█
val_loss,█▅▂▁▁
epoch,5
train_acc,0.88269
train_loss,0.03288
val_acc,0.875
val_loss,0.03594


wandb: Agent Starting Run: v63d2ms6 with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 3
wandb: 	optimizer: stochastic
wandb: 	weight_decay: 0.5
wandb: 	weight_init: random


...

Epoch 1: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 2: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 3: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 4: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 5: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 6: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 7: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 8: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 9: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 10: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▁▁▁▁▁▁▁▁▁
train_loss,█▇▆▅▄▃▂▂▁▁
val_acc,▁▁▁▁▁▁▁▁▁▁
val_loss,▅▃▁▁▁▂▃▄▆█
epoch,10
train_acc,0.09993
train_loss,0.23026
val_acc,0.1008
val_loss,0.23027


wandb: Agent Starting Run: 3vtonuiy with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_size: 32
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 4
wandb: 	optimizer: stochastic
wandb: 	weight_decay: 0.5
wandb: 	weight_init: random


Epoch 1: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 2: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 3: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 4: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 5: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303


epoch,▁▃▅▆█
train_acc,▁▁▁▁▁
train_loss,█▆▄▂▁
val_acc,▁▁▁▁▁
val_loss,▁▃▅▆█
epoch,5
train_acc,0.10078
train_loss,0.23026
val_acc,0.0914
val_loss,0.23027


#Question - 7

In [15]:
def plot_confusion_matrix(y_true, y_pred, config_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(10), yticklabels=range(10))
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(f'Confusion Matrix - {config_name}')

    wandb.log({f"Confusion Matrix - {config_name}": wandb.Image(plt)})
    plt.close()



def evaluate_best_configs(best_configs):
    x_train, y_train, x_val, y_val, x_test, y_test = Data_Preprocess()
    y_test_labels = np.argmax(y_test, axis=1)

    for config in best_configs:
        wandb.init(project="Vinod_Assignment1_Question7", name=f"Confusion_Matrix_{config['name']}", reinit=True)

        if config['optimizer'] == 'rmsprop':
            trained_weights = RMS_Opt(config['learning_rate'], x_train, y_train, x_val, y_val, config['epochs'], config['activation'], config['num_layers'], config['hidden_size'], config['weight_init'], config['batch_size'], 28*28, 10)
        elif config['optimizer'] == 'adam':
            trained_weights = Adam_Opt(config['learning_rate'], x_train, y_train, x_val, y_val, config['epochs'], config['activation'], config['num_layers'], config['hidden_size'], config['weight_init'], config['batch_size'], 28*28, 10)
        else:
            raise ValueError(f"Unsupported optimizer: {config['optimizer']}")

        y_pred_probs, _ = Forward_Propogation(x_test, trained_weights, config['num_layers'], config['activation'])
        y_pred_labels = np.argmax(y_pred_probs, axis=1)
        plot_confusion_matrix(y_test_labels, y_pred_labels, config['name'])
        wandb.finish()


if __name__ == "__main__":
    best_configs = [
    {
        'name': 'Best_Config_1',
        'epochs': 10,
        'num_layers': 5,
        'hidden_size': 128,
        'learning_rate': 0.001,
        'batch_size': 64,
        'optimizer': 'adam',
        'weight_decay': 0.5,
        'weight_init': 'xavier',
        'activation': 'tanh'
    },
    {
        'name': 'Best_Config_2',
        'epochs': 10,
        'num_layers': 4,
        'hidden_size': 64,
        'learning_rate': 0.001,
        'batch_size': 16,
        'optimizer': 'adam',
        'weight_decay': 0.0005,
        'weight_init': 'xavier',
        'activation': 'relu'
    }
]
evaluate_best_configs(best_configs)



Epoch 1: Train Acc=0.8551, Train Loss=0.0390, Val Acc=0.8534, Val Loss=0.0397
Epoch 2: Train Acc=0.8758, Train Loss=0.0339, Val Acc=0.8696, Val Loss=0.0353
Epoch 3: Train Acc=0.8834, Train Loss=0.0317, Val Acc=0.8754, Val Loss=0.0336
Epoch 4: Train Acc=0.8884, Train Loss=0.0303, Val Acc=0.8826, Val Loss=0.0327
Epoch 5: Train Acc=0.8925, Train Loss=0.0292, Val Acc=0.8858, Val Loss=0.0320
Epoch 6: Train Acc=0.8960, Train Loss=0.0283, Val Acc=0.8886, Val Loss=0.0316
Epoch 7: Train Acc=0.8983, Train Loss=0.0275, Val Acc=0.8884, Val Loss=0.0312
Epoch 8: Train Acc=0.9008, Train Loss=0.0268, Val Acc=0.8884, Val Loss=0.0310
Epoch 9: Train Acc=0.9027, Train Loss=0.0261, Val Acc=0.8886, Val Loss=0.0308
Epoch 10: Train Acc=0.9045, Train Loss=0.0256, Val Acc=0.8886, Val Loss=0.0307


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▄▅▆▆▇▇▇██
train_loss,█▅▄▃▃▂▂▂▁▁
val_acc,▁▄▅▇▇█████
val_loss,█▅▃▃▂▂▁▁▁▁
epoch,10
train_acc,0.90445
train_loss,0.02555
val_acc,0.8886
val_loss,0.03071


Epoch 1: Train Acc=0.8523, Train Loss=0.0412, Val Acc=0.8528, Val Loss=0.0408
Epoch 2: Train Acc=0.8686, Train Loss=0.0362, Val Acc=0.8706, Val Loss=0.0368
Epoch 3: Train Acc=0.8780, Train Loss=0.0334, Val Acc=0.8772, Val Loss=0.0348
Epoch 4: Train Acc=0.8823, Train Loss=0.0320, Val Acc=0.8798, Val Loss=0.0337
Epoch 5: Train Acc=0.8873, Train Loss=0.0307, Val Acc=0.8834, Val Loss=0.0328
Epoch 6: Train Acc=0.8910, Train Loss=0.0295, Val Acc=0.8852, Val Loss=0.0321
Epoch 7: Train Acc=0.8942, Train Loss=0.0288, Val Acc=0.8880, Val Loss=0.0316
Epoch 8: Train Acc=0.8969, Train Loss=0.0280, Val Acc=0.8898, Val Loss=0.0313
Epoch 9: Train Acc=0.8993, Train Loss=0.0272, Val Acc=0.8894, Val Loss=0.0310
Epoch 10: Train Acc=0.9016, Train Loss=0.0267, Val Acc=0.8904, Val Loss=0.0309


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▃▅▅▆▇▇▇██
train_loss,█▆▄▄▃▂▂▂▁▁
val_acc,▁▄▆▆▇▇████
val_loss,█▅▄▃▂▂▂▁▁▁
epoch,10
train_acc,0.90158
train_loss,0.02673
val_acc,0.8904
val_loss,0.03085


In [16]:
import wandb
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd

def plot_confusion_matrix(y_true, y_pred, config_name, class_names):
    """Logs an interactive confusion matrix to WandB."""
    cm = confusion_matrix(y_true, y_pred)

    # Log interactive Confusion Matrix
    wandb.log({
        f"Confusion Matrix - {config_name}": wandb.plot.confusion_matrix(
            probs=None,
            y_true=y_true,
            preds=y_pred,
            class_names=class_names
        )
    })


def evaluate_best_configs(best_configs):
    x_train, y_train, x_val, y_val, x_test, y_test = Data_Preprocess()
    y_test_labels = np.argmax(y_test, axis=1)

    class_names = ['Ankle boot', 'Bag', 'Coat', 'Dress', 'Pullover', 'Sandal', 'Shirt', 'Sneaker', 'T-shirt/top', 'Trouser']

    for config in best_configs:
        wandb.init(project="Vinod_Assignment1_Question7_A", name=f"Confusion_Matrix_{config['name']}", reinit=True)

        if config['optimizer'] == 'rmsprop':
            trained_weights = RMS_Opt(config['learning_rate'], x_train, y_train, x_val, y_val, config['epochs'], config['activation'], config['num_layers'], config['hidden_size'], config['weight_init'], config['batch_size'], 28*28, 10)
        elif config['optimizer'] == 'adam':
            trained_weights = Adam_Opt(config['learning_rate'], x_train, y_train, x_val, y_val, config['epochs'], config['activation'], config['num_layers'], config['hidden_size'], config['weight_init'], config['batch_size'], 28*28, 10)
        else:
            raise ValueError(f"Unsupported optimizer: {config['optimizer']}")

        y_pred_probs, _ = Forward_Propogation(x_test, trained_weights, config['num_layers'], config['activation'])
        y_pred_labels = np.argmax(y_pred_probs, axis=1)

        # Log interactive confusion matrix
        plot_confusion_matrix(y_test_labels, y_pred_labels, config['name'], class_names)

        wandb.finish()


if __name__ == "__main__":
    best_configs = [
    {
        'name': 'Best_Config_1',
        'epochs': 10,
        'num_layers': 5,
        'hidden_size': 128,
        'learning_rate': 0.001,
        'batch_size': 64,
        'optimizer': 'adam',
        'weight_decay': 0.5,
        'weight_init': 'xavier',
        'activation': 'tanh'
    },
    {
        'name': 'Best_Config_2',
        'epochs': 10,
        'num_layers': 4,
        'hidden_size': 64,
        'learning_rate': 0.001,
        'batch_size': 16,
        'optimizer': 'adam',
        'weight_decay': 0.0005,
        'weight_init': 'xavier',
        'activation': 'relu'
    }
]
evaluate_best_configs(best_configs)


Epoch 1: Train Acc=0.8550, Train Loss=0.0393, Val Acc=0.8504, Val Loss=0.0402
Epoch 2: Train Acc=0.8776, Train Loss=0.0336, Val Acc=0.8714, Val Loss=0.0352
Epoch 3: Train Acc=0.8849, Train Loss=0.0314, Val Acc=0.8784, Val Loss=0.0335
Epoch 4: Train Acc=0.8890, Train Loss=0.0301, Val Acc=0.8814, Val Loss=0.0325
Epoch 5: Train Acc=0.8924, Train Loss=0.0290, Val Acc=0.8824, Val Loss=0.0319
Epoch 6: Train Acc=0.8958, Train Loss=0.0281, Val Acc=0.8830, Val Loss=0.0314
Epoch 7: Train Acc=0.8990, Train Loss=0.0273, Val Acc=0.8844, Val Loss=0.0310
Epoch 8: Train Acc=0.9009, Train Loss=0.0267, Val Acc=0.8850, Val Loss=0.0307
Epoch 9: Train Acc=0.9027, Train Loss=0.0261, Val Acc=0.8854, Val Loss=0.0305
Epoch 10: Train Acc=0.9041, Train Loss=0.0255, Val Acc=0.8860, Val Loss=0.0304


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▄▅▆▆▇▇███
train_loss,█▅▄▃▃▂▂▂▁▁
val_acc,▁▅▇▇▇▇████
val_loss,█▄▃▃▂▂▁▁▁▁
epoch,10
train_acc,0.90407
train_loss,0.02553
val_acc,0.886
val_loss,0.0304


Epoch 1: Train Acc=0.8523, Train Loss=0.0424, Val Acc=0.8556, Val Loss=0.0416
Epoch 2: Train Acc=0.8651, Train Loss=0.0378, Val Acc=0.8658, Val Loss=0.0380
Epoch 3: Train Acc=0.8740, Train Loss=0.0348, Val Acc=0.8742, Val Loss=0.0360
Epoch 4: Train Acc=0.8804, Train Loss=0.0330, Val Acc=0.8774, Val Loss=0.0350
Epoch 5: Train Acc=0.8847, Train Loss=0.0317, Val Acc=0.8778, Val Loss=0.0344
Epoch 6: Train Acc=0.8886, Train Loss=0.0305, Val Acc=0.8800, Val Loss=0.0338
Epoch 7: Train Acc=0.8915, Train Loss=0.0294, Val Acc=0.8840, Val Loss=0.0333
Epoch 8: Train Acc=0.8946, Train Loss=0.0285, Val Acc=0.8842, Val Loss=0.0330
Epoch 9: Train Acc=0.8964, Train Loss=0.0280, Val Acc=0.8850, Val Loss=0.0328
Epoch 10: Train Acc=0.8984, Train Loss=0.0273, Val Acc=0.8862, Val Loss=0.0326


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▃▄▅▆▇▇▇██
train_loss,█▆▄▄▃▂▂▂▁▁
val_acc,▁▃▅▆▆▇▇███
val_loss,█▅▄▃▂▂▂▁▁▁
epoch,10
train_acc,0.89842
train_loss,0.02732
val_acc,0.8862
val_loss,0.03264


#Question - 8

In [28]:
def main():
    sweep_config = {
        'method': 'bayes',
        'metric': {'name': 'accuracy', 'goal': 'maximize'},
        'parameters': {
            'epochs': {'values': [5, 10]},
            'num_layers': {'values': [3, 4, 5]},
            'hidden_size': {'values': [32, 64, 128]},
            'weight_decay': {'values': [0, 0.0005, 0.5]},
            'learning_rate': {'values': [1e-3, 1e-4]},
            'optimizer': {'values': ['stochastic', 'momentum', 'nag', 'rmsprop', 'adam', 'nadam']},
            'batch_size': {'values': [16, 32, 64]},
            'weight_init': {'values': ['random', 'xavier']},
            'activation': {'values': ['sigmoid', 'tanh', 'relu']},
        }
    }
    sweep_id = wandb.sweep(sweep_config, project="Vinod_Assignment1_Question8_mse")
    wandb.agent(sweep_id, function=train, count=50)

def train():
    wandb.init(project="Vinod_Assignment1_Question8_mse")
    # wandb.init()
    config = wandb.config
    run_name = f"Opt-{config.optimizer}_Layers-{config.num_layers}_HS-{config.hidden_size}_LR-{config.learning_rate}_Batch-{config.batch_size}_Act-{config.activation}"
    wandb.run.name = run_name

    # x_train, y_train, x_val, y_val, _, _ = Data_Preprocess()

    optimizer = config.optimizer

    if optimizer == 'stochastic':
        trained_weights = Stochastic_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='mse')
        # trained_weights = Stochastic_GD(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size)
    elif optimizer == 'momentum':
        trained_weights = Momentum_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='mse')
    elif optimizer == 'nag':
        trained_weights = Nesterov_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='mse')
    elif optimizer == 'rmsprop':
        trained_weights = RMS_Opt(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='mse')
    elif optimizer == 'adam':
        trained_weights = Adam_Opt(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='mse')
    elif optimizer == 'nadam':
        trained_weights = NAdam_Opt(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='mse')

    #wandb.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss})

    wandb.finish()

if __name__ == "__main__":
    main()


Create sweep with ID: 106csc1h
Sweep URL: https://wandb.ai/viinod9-iitm/Vinod_Assignment1_Question8_mse/sweeps/106csc1h


wandb: Agent Starting Run: 3z0xbc30 with config:
wandb: 	activation: tanh
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.0001
wandb: 	num_layers: 4
wandb: 	optimizer: nag
wandb: 	weight_decay: 0
wandb: 	weight_init: xavier


Epoch 1: Train Acc=0.6337, Train Loss=0.0626, Val Acc=0.6424, Val Loss=0.0621
Epoch 2: Train Acc=0.6799, Train Loss=0.0494, Val Acc=0.6870, Val Loss=0.0487
Epoch 3: Train Acc=0.7142, Train Loss=0.0429, Val Acc=0.7236, Val Loss=0.0422
Epoch 4: Train Acc=0.7395, Train Loss=0.0390, Val Acc=0.7466, Val Loss=0.0383
Epoch 5: Train Acc=0.7579, Train Loss=0.0361, Val Acc=0.7660, Val Loss=0.0353


epoch,▁▃▅▆█
train_acc,▁▄▆▇█
train_loss,█▅▃▂▁
val_acc,▁▄▆▇█
val_loss,█▄▃▂▁
epoch,5
train_acc,0.75795
train_loss,0.03607
val_acc,0.766
val_loss,0.03533


wandb: Agent Starting Run: sqa0hrdz with config:
wandb: 	activation: relu
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 5
wandb: 	optimizer: nag
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: random


...

Epoch 1: Train Acc=0.1198, Train Loss=0.0899, Val Acc=0.1194, Val Loss=0.0899
Epoch 2: Train Acc=0.5901, Train Loss=0.0551, Val Acc=0.5988, Val Loss=0.0545
Epoch 3: Train Acc=0.7320, Train Loss=0.0374, Val Acc=0.7370, Val Loss=0.0369
Epoch 4: Train Acc=0.7698, Train Loss=0.0312, Val Acc=0.7764, Val Loss=0.0309
Epoch 5: Train Acc=0.8003, Train Loss=0.0271, Val Acc=0.8082, Val Loss=0.0266
Epoch 6: Train Acc=0.8349, Train Loss=0.0239, Val Acc=0.8376, Val Loss=0.0236
Epoch 7: Train Acc=0.8468, Train Loss=0.0221, Val Acc=0.8466, Val Loss=0.0220
Epoch 8: Train Acc=0.8551, Train Loss=0.0209, Val Acc=0.8524, Val Loss=0.0209
Epoch 9: Train Acc=0.8623, Train Loss=0.0198, Val Acc=0.8582, Val Loss=0.0200
Epoch 10: Train Acc=0.8684, Train Loss=0.0189, Val Acc=0.8634, Val Loss=0.0193


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▅▇▇▇█████
train_loss,█▅▃▂▂▁▁▁▁▁
val_acc,▁▆▇▇▇█████
val_loss,█▄▃▂▂▁▁▁▁▁
epoch,10
train_acc,0.86836
train_loss,0.01894
val_acc,0.8634
val_loss,0.01934


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: r0f1s10k with config:
wandb: 	activation: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 3
wandb: 	optimizer: momentum
wandb: 	weight_decay: 0
wandb: 	weight_init: random


Epoch 1: Train Acc=0.1008, Train Loss=0.0900, Val Acc=0.0914, Val Loss=0.0900
Epoch 2: Train Acc=0.1085, Train Loss=0.0899, Val Acc=0.1006, Val Loss=0.0899
Epoch 3: Train Acc=0.2599, Train Loss=0.0780, Val Acc=0.2692, Val Loss=0.0779
Epoch 4: Train Acc=0.5786, Train Loss=0.0523, Val Acc=0.5886, Val Loss=0.0517
Epoch 5: Train Acc=0.6817, Train Loss=0.0428, Val Acc=0.6846, Val Loss=0.0422


epoch,▁▃▅▆█
train_acc,▁▁▃▇█
train_loss,██▆▂▁
val_acc,▁▁▃▇█
val_loss,██▆▂▁
epoch,5
train_acc,0.68173
train_loss,0.04279
val_acc,0.6846
val_loss,0.04221


In [29]:
def main():
    sweep_config = {
        'method': 'bayes',
        'metric': {'name': 'accuracy', 'goal': 'maximize'},
        'parameters': {
            'epochs': {'values': [5, 10]},
            'num_layers': {'values': [3, 4, 5]},
            'hidden_size': {'values': [32, 64, 128]},
            'weight_decay': {'values': [0, 0.0005, 0.5]},
            'learning_rate': {'values': [1e-3, 1e-4]},
            'optimizer': {'values': ['stochastic', 'momentum', 'nag', 'rmsprop', 'adam', 'nadam']},
            'batch_size': {'values': [16, 32, 64]},
            'weight_init': {'values': ['random', 'xavier']},
            'activation': {'values': ['sigmoid', 'tanh', 'relu']},
        }
    }
    sweep_id = wandb.sweep(sweep_config, project="Vinod_Assignment1_Question8_CrossEntropy")
    wandb.agent(sweep_id, function=train, count=50)

def train():
    wandb.init(project="Vinod_Assignment1_Question8_CrossEntropy")
    # wandb.init()
    config = wandb.config
    run_name = f"Opt-{config.optimizer}_Layers-{config.num_layers}_HS-{config.hidden_size}_LR-{config.learning_rate}_Batch-{config.batch_size}_Act-{config.activation}"
    wandb.run.name = run_name

    # x_train, y_train, x_val, y_val, _, _ = Data_Preprocess()

    optimizer = config.optimizer

    if optimizer == 'stochastic':
        trained_weights = Stochastic_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='mscross_entropy')
        # trained_weights = Stochastic_GD(lr, x_train, y_train, x_val, y_val, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size)
    elif optimizer == 'momentum':
        trained_weights = Momentum_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')
    elif optimizer == 'nag':
        trained_weights = Nesterov_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')
    elif optimizer == 'rmsprop':
        trained_weights = RMS_Opt(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')
    elif optimizer == 'adam':
        trained_weights = Adam_Opt(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')
    elif optimizer == 'nadam':
        trained_weights = NAdam_Opt(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='cross_entropy')

    #wandb.log({"train_acc": train_acc, "train_loss": train_loss, "val_acc": val_acc, "val_loss": val_loss})

    wandb.finish()

if __name__ == "__main__":
    main()


Create sweep with ID: uvbda342
Sweep URL: https://wandb.ai/viinod9-iitm/Vinod_Assignment1_Question8_CrossEntropy/sweeps/uvbda342


wandb: Agent Starting Run: 9ggby5fx with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	num_layers: 4
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0
wandb: 	weight_init: random


Epoch 1: Train Acc=0.1988, Train Loss=0.2227, Val Acc=0.1914, Val Loss=0.2227
Epoch 2: Train Acc=0.1996, Train Loss=0.1854, Val Acc=0.2020, Val Loss=0.1853
Epoch 3: Train Acc=0.1992, Train Loss=0.1748, Val Acc=0.2012, Val Loss=0.1746
Epoch 4: Train Acc=0.2016, Train Loss=0.1712, Val Acc=0.2004, Val Loss=0.1710
Epoch 5: Train Acc=0.2023, Train Loss=0.1695, Val Acc=0.2010, Val Loss=0.1693
Epoch 6: Train Acc=0.2031, Train Loss=0.1687, Val Acc=0.2014, Val Loss=0.1684
Epoch 7: Train Acc=0.2037, Train Loss=0.1682, Val Acc=0.2022, Val Loss=0.1680
Epoch 8: Train Acc=0.2042, Train Loss=0.1679, Val Acc=0.2028, Val Loss=0.1677
Epoch 9: Train Acc=0.2048, Train Loss=0.1676, Val Acc=0.2032, Val Loss=0.1675
Epoch 10: Train Acc=0.2054, Train Loss=0.1674, Val Acc=0.2046, Val Loss=0.1674


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁▂▁▄▅▆▆▇▇█
train_loss,█▃▂▁▁▁▁▁▁▁
val_acc,▁▇▆▆▆▆▇▇▇█
val_loss,█▃▂▁▁▁▁▁▁▁
epoch,10
train_acc,0.20536
train_loss,0.16743
val_acc,0.2046
val_loss,0.16735


wandb: Agent Starting Run: 8oygmsim with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 5
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 3
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: xavier


...

Epoch 1: Train Acc=0.0999, Train Loss=0.2303, Val Acc=0.1008, Val Loss=0.2303
Epoch 2: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 3: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 4: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 5: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 6: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 7: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 8: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 9: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303
Epoch 10: Train Acc=0.1008, Train Loss=0.2303, Val Acc=0.0914, Val Loss=0.2303


epoch,▁▂▃▃▄▅▆▆▇█
train_acc,▁█████████
train_loss,█▁▁▁▁▁▁▁▁▁
val_acc,█▁▁▁▁▁▁▁▁▁
val_loss,▁▆████████
epoch,10
train_acc,0.10078
train_loss,0.23026
val_acc,0.0914
val_loss,0.23028


wandb: Agent Starting Run: rzyg2q6v with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	epochs: 10
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.0001
wandb: 	num_layers: 3
wandb: 	optimizer: stochastic
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: xavier


wandb: ERROR Run rzyg2q6v errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-29-756b65a1eb69>", line 32, in train
wandb: ERROR     trained_weights = Stochastic_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='mscross_entropy')
wandb: ERROR                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "<ipython-input-20-6bea02f99859>", line 142, in Stochastic_GD
wandb: ERROR     print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Train Loss={train_loss:.4f}, Val

wandb: ERROR Run zom10e84 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-29-756b65a1eb69>", line 32, in train
wandb: ERROR     trained_weights = Stochastic_GD(config.learning_rate, x_train, y_train, x_val, y_val, config.epochs, config.activation, config.num_layers, config.hidden_size, config.weight_init, config.batch_size, 28*28, 10, loss_function='mscross_entropy')
wandb: ERROR                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "<ipython-input-20-6bea02f99859>", line 142, in Stochastic_GD
wandb: ERROR     print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Train Loss={train_loss:.4f}, Val

#Question- 10

In [17]:
def Data_Preprocess_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Split into train and validation
    val_size = 5000
    x_val, y_val = x_train[:val_size], y_train[:val_size]
    x_train, y_train = x_train[val_size:], y_train[val_size:]

    # Normalize dataset
    x_train, x_val, x_test = x_train / 255.0, x_val / 255.0, x_test / 255.0

    # One-hot encoding
    y_train = to_categorical(y_train, 10)
    y_val = to_categorical(y_val, 10)
    y_test = to_categorical(y_test, 10)

    return x_train, y_train, x_val, y_val, x_test, y_test

x_train_mnist, y_train_mnist, x_val_mnist, y_val_mnist, x_test_mnist, y_test_mnist = Data_Preprocess_mnist()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [19]:
# Set parameters and train
num_hidden_layer = 5
num_nodes_hidden_layers = [128]
weight = 'xavier'
input_size = 28 * 28  # Flattened image size
output_size = 10  # Number of classes
lr = 0.001
batch_size = 64
epochs = 10
activation = 'tanh'

trained_weights_mnist1 = Adam_Opt(lr, x_train_mnist, y_train_mnist, x_val_mnist, y_val_mnist, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, beta1=0.9, beta2=0.999, epsilon=1e-8, loss_function='cross_entropy')


Epoch 1: Train Acc=0.9457, Train Loss=0.0179, Val Acc=0.9446, Val Loss=0.0175
Epoch 2: Train Acc=0.9638, Train Loss=0.0121, Val Acc=0.9604, Val Loss=0.0128
Epoch 3: Train Acc=0.9716, Train Loss=0.0096, Val Acc=0.9668, Val Loss=0.0110
Epoch 4: Train Acc=0.9763, Train Loss=0.0081, Val Acc=0.9698, Val Loss=0.0101
Epoch 5: Train Acc=0.9795, Train Loss=0.0070, Val Acc=0.9710, Val Loss=0.0095
Epoch 6: Train Acc=0.9816, Train Loss=0.0061, Val Acc=0.9734, Val Loss=0.0092
Epoch 7: Train Acc=0.9839, Train Loss=0.0054, Val Acc=0.9734, Val Loss=0.0090
Epoch 8: Train Acc=0.9859, Train Loss=0.0048, Val Acc=0.9748, Val Loss=0.0088
Epoch 9: Train Acc=0.9875, Train Loss=0.0042, Val Acc=0.9756, Val Loss=0.0087
Epoch 10: Train Acc=0.9886, Train Loss=0.0037, Val Acc=0.9764, Val Loss=0.0087


In [20]:
# Set parameters and train
num_hidden_layer = 4
num_nodes_hidden_layers = [64]
weight = 'xavier'
input_size = 28 * 28  # Flattened image size
output_size = 10  # Number of classes
lr = 0.001
batch_size = 16
epochs = 10
activation = 'relu'

trained_weights_mnist2 = Adam_Opt(lr, x_train_mnist, y_train_mnist, x_val_mnist, y_val_mnist, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, beta1=0.9, beta2=0.999, epsilon=1e-8, loss_function='cross_entropy')


Epoch 1: Train Acc=0.9460, Train Loss=0.0177, Val Acc=0.9498, Val Loss=0.0175
Epoch 2: Train Acc=0.9657, Train Loss=0.0113, Val Acc=0.9654, Val Loss=0.0121
Epoch 3: Train Acc=0.9740, Train Loss=0.0087, Val Acc=0.9712, Val Loss=0.0105
Epoch 4: Train Acc=0.9776, Train Loss=0.0073, Val Acc=0.9724, Val Loss=0.0099
Epoch 5: Train Acc=0.9800, Train Loss=0.0064, Val Acc=0.9732, Val Loss=0.0098
Epoch 6: Train Acc=0.9813, Train Loss=0.0060, Val Acc=0.9734, Val Loss=0.0101
Epoch 7: Train Acc=0.9829, Train Loss=0.0053, Val Acc=0.9738, Val Loss=0.0099
Epoch 8: Train Acc=0.9842, Train Loss=0.0048, Val Acc=0.9718, Val Loss=0.0100
Epoch 9: Train Acc=0.9849, Train Loss=0.0045, Val Acc=0.9706, Val Loss=0.0102
Epoch 10: Train Acc=0.9855, Train Loss=0.0043, Val Acc=0.9702, Val Loss=0.0106


In [21]:
# Set parameters and train
num_hidden_layer = 3
num_nodes_hidden_layers = [64]
weight = 'xavier'
input_size = 28 * 28
output_size = 10
lr = 0.001
batch_size = 32
epochs = 10
activation = 'tanh'

trained_weights_mnist3 = Adam_Opt(lr, x_train_mnist, y_train_mnist, x_val_mnist, y_val_mnist, epochs, activation, num_hidden_layer, num_nodes_hidden_layers, weight, batch_size, input_size, output_size, beta1=0.9, beta2=0.999, epsilon=1e-8, loss_function='cross_entropy')


Epoch 1: Train Acc=0.9455, Train Loss=0.0187, Val Acc=0.9470, Val Loss=0.0180
Epoch 2: Train Acc=0.9592, Train Loss=0.0140, Val Acc=0.9600, Val Loss=0.0139
Epoch 3: Train Acc=0.9666, Train Loss=0.0115, Val Acc=0.9642, Val Loss=0.0121
Epoch 4: Train Acc=0.9715, Train Loss=0.0099, Val Acc=0.9672, Val Loss=0.0111
Epoch 5: Train Acc=0.9749, Train Loss=0.0087, Val Acc=0.9684, Val Loss=0.0104
Epoch 6: Train Acc=0.9778, Train Loss=0.0077, Val Acc=0.9698, Val Loss=0.0100
Epoch 7: Train Acc=0.9799, Train Loss=0.0070, Val Acc=0.9710, Val Loss=0.0097
Epoch 8: Train Acc=0.9816, Train Loss=0.0063, Val Acc=0.9708, Val Loss=0.0094
Epoch 9: Train Acc=0.9833, Train Loss=0.0058, Val Acc=0.9724, Val Loss=0.0093
Epoch 10: Train Acc=0.9848, Train Loss=0.0053, Val Acc=0.9714, Val Loss=0.0092
